# <strong>SQL Queries for USDA Production Data Analysis</strong>

This notebook contains SQL queries to analyze the USDA production database. 
Make sure to have the `USDA_production.sqlite` database file in the same directory as this notebook.

<strong>I recommend using "DB Browser" to explore and interact with the database. It's simple to create tables and efficiently view and retrieve data.</strong>

### <strong>Instructions:</strong>
1. Execute the cells step by step to run the queries.
2. Read the explanations in the markdown cells for insights into each query's purpose.
3. Modify the queries if needed to adapt them to specific scenarios.
 
### <strong>Scenario:</strong>
Data Scientist at USDA (United States Department of Agriculture)

### <strong>Context:</strong>
You are a Data Scientist working at the USDA. Your department has been tracking the production of various agricultural commodities across different states. 

Your datasets include:

`milk_production`, `cheese_production`, `coffee_production`, `honey_production`, `yogurt_production`, and a `state_lookup` table. 

The data spans multiple years and states, with varying levels of production for each commodity.

Your manager has requested that you generate insights from this data to aid in future planning and decision-making. You'll need to use SQL queries to answer the questions that come up in meetings, reports, or strategic discussions.

### <strong>Objectives:</strong>
Assess state-by-state production for each commodity.

Identify trends or anomalies.

Offer data-backed suggestions for areas that may need more attention.


NOTE: All answer entries are numeric and only numbers and periods. The autograder does not accept commas for the final project.

In [ ]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Connect to the USDA production database
conn = sqlite3.connect('USDA_production.sqlite')
cursor = conn.cursor()
print('Database connection established.')

#### <strong>Question 1</Strong> 
Can you find out the total milk production for 2023? Your manager wants this information for the yearly report.

What is the total milk production for 2023?.

In [ ]:
# SQL Query to calculate the total milk production for the year 2023.
query = """
SELECT Year, SUM(Value) AS Total_Value
FROM milk_production
WHERE Year = 2023;
"""

df = pd.read_sql_query(query, conn)
df

#### <strong>Question 2</strong>
Which states had cheese production greater than 100 million in April 2023? The Cheese Department wants to focus their marketing efforts there. 

How many states are there?

In [ ]:
# SQL Query to retrieve cheese production values greater than 100 million in 2023,
# including state names and ANSI codes for better context.
query = """
SELECT c.Year, c.Period, c.State_ANSI, s.State, c.Value
FROM state_lookup s
JOIN cheese_production c on c.State_ANSI = s.State_ANSI
WHERE c.Value > 100000000 AND c.Year = 2023 AND c.Period = 'APR';
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 3</strong>
Your manager wants to know how coffee production has changed over the years. 

What is the total value of coffee production for 2011?

In [ ]:
# SQL Query to calculate the total coffee production for the year 2011.
query = """
SELECT SUM(Value) AS Total_Value
FROM coffee_production
WHERE Year = 2011;
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 4</strong>
There's a meeting with the Honey Council next week. Find the average honey production for 2022 so you're prepared.

In [ ]:
# SQL Query to calculate the average honey production for the year 2022.
query = """
SELECT AVG(Value)
FROM honey_production
WHERE Year = 2022;
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 5</strong> 
The State Relations team wants a list of all states names with their corresponding ANSI codes. Can you generate that list?

What is the State_ANSI code for Florida?

In [ ]:
# SQL Query to retrieve the ANSI code for the state of Florida.
query = """
SELECT State, State_ANSI
FROM state_lookup
WHERE State = "FLORIDA";
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 6</strong>
For a cross-commodity report, can you list all states with their cheese production values, even if they didn't produce any cheese in April of 2023?

What is the total for NEW JERSEY?

In [ ]:
# SQL Query to find the total cheese production in New Jersey in April 2023.
query = """
SELECT s.State, s.State_ANSI, SUM(c.Value) AS Total_Value
FROM state_lookup s
JOIN cheese_production c on s.State_ANSI = c.State_ANSI
WHERE State = "NEW JERSEY" AND c.Year = 2023 
AND c.Period='APR';
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 7</strong>
Can you find the total yogurt production for states in the year 2022 which also have cheese production data from 2023? This will help the Dairy Division in their planning.

In [ ]:
# SQL Query to calculate the total yogurt production in 2022
# for states that also have cheese production data for 2022.
query = """
SELECT sum(y.Value) AS Total_Value
FROM yogurt_production y 
WHERE y.State_ANSI IN (
SELECT DISTINCT c.State_ANSI 
FROM cheese_production c 
WHERE c.Year = 2023) AND y.Year = 2022; 
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 8</strong>
List all states from state_lookup that are missing from milk_production in 2023.

How many states are there?

In [ ]:
# SQL Query to list states that do not appear in the milk production data for 2023.
query = """
SELECT count(s.State) AS Total_states
FROM state_lookup s
WHERE s.State_ANSI NOT IN (
	SELECT DISTINCT m.State_ANSI 
	FROM milk_production m 
	WHERE m.Year = 2023);
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 9</strong>
List all states with their cheese production values, including states that didn't produce any cheese in April 2023.

Did Delaware produce any cheese in April 2023?

In [ ]:
# SQL Query to check if Delaware produced cheese in 2023.
query = """
SELECT s.State, s.State_ANSI, c.Value, c.Year
FROM state_lookup s
JOIN cheese_production c on s.State_ANSI = c.State_ANSI
WHERE c.Year = 2023 AND s.State = 'DELAWARE';
"""
df = pd.read_sql_query(query, conn)
df

#### <strong>Question 10</strong>
Find the average coffee production for all years where the honey production exceeded 1 million.

In [ ]:
# SQL Query to find the average coffee production for all years
# where total honey production exceeded 1 million.
query = """
SELECT AVG(c.Value) AS Average_Value
FROM coffee_production c
WHERE c.Year IN (
    SELECT h.Year
    FROM honey_production h
    GROUP BY h.Year
    HAVING SUM(h.Value) > 1000000);
"""
df = pd.read_sql_query(query, conn)
df

## Closing the Database Connection
After completing the queries, close the database connection.

In [ ]:
# Close the database connection
conn.close()
print('Database connection closed.')